In [1]:
import os
import sys
import re
import time
import random
import warnings
import collections
from tqdm import tqdm
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import seaborn as sns

sys.path.append('../../src')
import cb_utils

sns.set(style="darkgrid")
pd.options.display.max_columns = 500

%load_ext autoreload
%autoreload 2

# Milliman MANI File ETL

### Config

In [2]:
data_dir = '/Users/bp/workspace/msh/milliman_data'
current_folder = '20240812'
# pro_file_name = 'ACO Builder MSSP Prospective Explorer v2025.3.txt'
# retro_file_name = 'ACO Builder MSSP Retrospective Explorer v2025.3.txt'
table_prefix = 'milliman_mani'

### DataTables

In [6]:
file_name = 'Medicare Advantage Network Intelligence (MANI) CY2020 Data Tables - v2024.0 - Nationwide.csv'
year = 2020
mani_data_tables_2020_df = pd.read_csv(f'{data_dir}/{current_folder}/{file_name}',  encoding='latin1', low_memory=False)
# display(mani_data_tables_2020_df.head())
mani_data_tables_2020_df = cb_utils.df_format_columns(mani_data_tables_2020_df)
mani_data_tables_2020_df.head()

,Year,ACO_Builder_ID,TIN_CCN_Type,TIN_CCN_ID,Provider_Name,PAC_ID,St_MSA,Carrier,PY,Risk_Score_MMWtd,Risk_Score_MMWtd_Region,MA_Benchmark_Rate,MA_Bid_Amount,Paid_atFFSReimb,Paid_atFFSReimb_Region,IP_Admits,IP_Admits_Imputed,IP_Admits_Region,IP_Days,IP_Days_Imputed,IP_Days_Region,SNF_Admits,SNF_Admits_Imputed,SNF_Admits_Region,SNF_Days,SNF_Days_Imputed,SNF_Days_Region,ED,ED_Imputed,ED_Region,Phy_Office,Phy_Office_Imputed,Phy_Office_Region,Rad_Adv_Imag_OP,Rad_Adv_Imag_OP_Imputed,Rad_Adv_Imag_OP_Region,Rad_Adv_Imag_NonHosp,Rad_Adv_Imag_NonHosp_Imputed,Rad_Adv_Imag_NonHosp_Region,Rad_Gen_OP,Rad_Gen_OP_Imputed,Rad_Gen_OP_Region,Rad_Gen_NonHosp,Rad_Gen_NonHosp_Imputed,Rad_Gen_NonHosp_Region,Rad_Therap_OP,Rad_Therap_OP_Imputed,Rad_Therap_OP_Region,Rad_Therap_NonHosp,Rad_Therap_NonHosp_Imputed,Rad_Therap_NonHosp_Region,Prev_Physicals,Prev_Physicals_Imputed,Prev_Physicals_Region,Surgery_ASC,Surgery_ASC_Imputed,Surgery_ASC_Region,Surgery_OP,Surgery_OP_Imputed,Surgery_OP_Region,Part_B_Rx_NonHosp,Part_B_Rx_NonHosp_Imputed,Part_B_Rx_NonHosp_Region,Part_B_Rx_OP,Part_B_Rx_OP_Imputed,Part_B_Rx_OP_Region,Lab_NonHosp,Lab_NonHosp_Imputed,Lab_NonHosp_Region,Lab_OP,Lab_OP_Imputed,Lab_OP_Region
0,2020,00004B7C4023,TIN,474454561.0,RINDFLEISCH FAMILY PRACTICE PLLC,9335446202,ID-26820,"Blue Cross of Idaho Health Services, Inc.",70.41667,978.40233,940.96367,9.102015e+05,813222.59670,8.487632e+05,7.842224e+05,17.18238,17.18238,16.17895,95.00000,0.00000,91.40048,0.00000,0.00000,2.90345,0.0,0.0,78.62068,66.00000,0.00000,38.43124,672.25134,2.25134,531.47002,41.00000,0.00000,49.06599,7.44854,7.44854,11.09374,101.00000,0.00000,78.88074,67.00000,0.00000,66.09894,23.63697,5.63697,6.92966,0.0,0.0,2.13013,23.53506,4.53506,37.11633,5.58672,5.58672,7.48148,26.00000,0.00000,35.62701,102.0,2.0,67.61381,44.0,1.0,79.75242,1065.00000,0.00000,778.46228,280.00000,0.00000,535.58037
1,2020,00004B7C4023,TIN,474454561.0,RINDFLEISCH FAMILY PRACTICE PLLC,9335446202,ID-26820,FFS,139.00000,1738.72311,1806.87182,1.617316e+06,0.00000,1.544586e+06,1.593759e+06,44.55842,13.55842,0.00000,223.42427,5.42427,0.00000,4.10977,4.10977,0.00000,148.0,0.0,0.00000,62.00000,0.00000,0.00000,1098.00000,0.00000,0.00000,71.00000,0.00000,0.00000,11.04066,11.04066,0.00000,103.00000,0.00000,0.00000,91.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.0,0.0,0.00000,12.97964,12.97964,0.00000,18.05664,1.05664,0.00000,38.48064,7.48064,0.00000,163.0,0.0,0.00000,49.0,1.0,0.00000,2715.00000,0.00000,0.00000,286.49336,5.49336,0.00000
2,2020,00004B7C4023,TIN,474454561.0,RINDFLEISCH FAMILY PRACTICE PLLC,9335446202,ID-26820,Humana Inc.,3.33333,37.09248,34.95186,3.453604e+04,30321.35652,4.988403e+04,2.742271e+04,0.00000,0.00000,0.53436,0.00000,0.00000,2.25320,0.00000,0.00000,0.06985,0.0,0.0,1.51062,2.93185,2.93185,1.12610,39.00000,0.00000,20.82886,1.17593,1.17593,1.91406,1.00000,1.00000,0.50789,1.00000,1.00000,2.79236,3.17279,3.17279,2.83264,0.00000,0.00000,0.32776,0.0,0.0,0.05884,0.00000,0.00000,1.81274,0.00000,0.00000,0.30693,11.00000,11.00000,1.58325,1.0,1.0,3.02750,0.0,0.0,2.70668,57.00000,0.00000,31.71387,2.45180,2.45180,19.91043
3,2020,00004B7C4023,TIN,474454561.0,RINDFLEISCH FAMILY PRACTICE PLLC,9335446202,ID-26820,ID Other Carriers,1.00000,8.23925,7.80952,7.656489e+03,6593.75279,3.704882e+03,5.740482e+03,0.00000,0.00000,0.10484,0.00000,0.00000,0.46342,0.00000,0.00000,0.01048,0.0,0.0,0.30821,0.00000,0.00000,0.26422,12.46735,12.46735,5.23755,1.12693,1.12693,0.46967,0.00000,0.00000,0.12790,1.49576,1.49576,0.59546,1.00000,1.00000,0.79047,0.00000,0.00000,0.02307,0.0,0.0,0.04823,1.00000,1.00000,0.59125,0.00000,0.00000,0.13000,0.00000,0.00000,0.40677,1.0,1.0,0.61840,1.0,1.0,0.40470,9.98463,9.98463,8.83154,0.00000,0.00000,5.51019
4,2020,00004B7C4023,TIN,474454561.0,RINDFLEISCH FAMILY PRACTICE PLLC,9335446202,ID-26820,"Molina Healthcare, Inc.",5.00000,57.99166,61.51546,5.388991e+04,49180.45071,9.815472e+04,7.204374e+04,1.00000,1.00000,1.41137,3.85948,3.85948,7.58740,1.00000,1.00000,0.23034,13.0,0.0,6.15417,2.66142,2.

,year,aco_builder_id,tin_ccn_type,tin_ccn_id,provider_name,pac_id,st_msa,carrier,py,risk_score_mmwtd,risk_score_mmwtd_region,ma_benchmark_rate,ma_bid_amount,paid_atffsreimb,paid_atffsreimb_region,ip_admits,ip_admits_imputed,ip_admits_region,ip_days,ip_days_imputed,ip_days_region,snf_admits,snf_admits_imputed,snf_admits_region,snf_days,snf_days_imputed,snf_days_region,ed,ed_imputed,ed_region,phy_office,phy_office_imputed,phy_office_region,rad_adv_imag_op,rad_adv_imag_op_imputed,rad_adv_imag_op_region,rad_adv_imag_nonhosp,rad_adv_imag_nonhosp_imputed,rad_adv_imag_nonhosp_region,rad_gen_op,rad_gen_op_imputed,rad_gen_op_region,rad_gen_nonhosp,rad_gen_nonhosp_imputed,rad_gen_nonhosp_region,rad_therap_op,rad_therap_op_imputed,rad_therap_op_region,rad_therap_nonhosp,rad_therap_nonhosp_imputed,rad_therap_nonhosp_region,prev_physicals,prev_physicals_imputed,prev_physicals_region,surgery_asc,surgery_asc_imputed,surgery_asc_region,surgery_op,surgery_op_imputed,surgery_op_region,part_b_rx_nonhosp,part_b_rx_nonhosp_imputed,part_b_rx_nonhosp_region,part_b_rx_op,part_b_rx_op_imputed,part_b_rx_op_region,lab_nonhosp,lab_nonhosp_imputed,lab_nonhosp_region,lab_op,lab_op_imputed,lab_op_region
0,2020,00004B7C4023,TIN,474454561.0,RINDFLEISCH FAMILY PRACTICE PLLC,9335446202,ID-26820,"Blue Cross of Idaho Health Services, Inc.",70.41667,978.40233,940.96367,9.102015e+05,813222.59670,8.487632e+05,7.842224e+05,17.18238,17.18238,16.17895,95.00000,0.00000,91.40048,0.00000,0.00000,2.90345,0.0,0.0,78.62068,66.00000,0.00000,38.43124,672.25134,2.25134,531.47002,41.00000,0.00000,49.06599,7.44854,7.44854,11.09374,101.00000,0.00000,78.88074,67.00000,0.00000,66.09894,23.63697,5.63697,6.92966,0.0,0.0,2.13013,23.53506,4.53506,37.11633,5.58672,5.58672,7.48148,26.00000,0.00000,35.62701,102.0,2.0,67.61381,44.0,1.0,79.75242,1065.00000,0.00000,778.46228,280.00000,0.00000,535.58037
1,2020,00004B7C4023,TIN,474454561.0,RINDFLEISCH FAMILY PRACTICE PLLC,9335446202,ID-26820,FFS,139.00000,1738.72311,1806.87182,1.617316e+06,0.00000,1.544586e+06,1.593759e+06,44.55842,13.55842,0.00000,223.42427,5.42427,0.00000,4.10977,4.10977,0.00000,148.0,0.0,0.00000,62.00000,0.00000,0.00000,1098.00000,0.00000,0.00000,71.00000,0.00000,0.00000,11.04066,11.04066,0.00000,103.00000,0.00000,0.00000,91.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.0,0.0,0.00000,12.97964,12.97964,0.00000,18.05664,1.05664,0.00000,38.48064,7.48064,0.00000,163.0,0.0,0.00000,49.0,1.0,0.00000,2715.00000,0.00000,0.00000,286.49336,5.49336,0.00000
2,2020,00004B7C4023,TIN,474454561.0,RINDFLEISCH FAMILY PRACTICE PLLC,9335446202,ID-26820,Humana Inc.,3.33333,37.09248,34.95186,3.453604e+04,30321.35652,4.988403e+04,2.742271e+04,0.00000,0.00000,0.53436,0.00000,0.00000,2.25320,0.00000,0.00000,0.06985,0.0,0.0,1.51062,2.93185,2.93185,1.12610,39.00000,0.00000,20.82886,1.17593,1.17593,1.91406,1.00000,1.00000,0.50789,1.00000,1.00000,2.79236,3.17279,3.17279,2.83264,0.00000,0.00000,0.32776,0.0,0.0,0.05884,0.00000,0.00000,1.81274,0.00000,0.00000,0.30693,11.00000,11.00000,1.58325,1.0,1.0,3.02750,0.0,0.0,2.70668,57.00000,0.00000,31.71387,2.45180,2.45180,19.91043
3,2020,00004B7C4023,TIN,474454561.0,RINDFLEISCH FAMILY PRACTICE PLLC,9335446202,ID-26820,ID Other Carriers,1.00000,8.23925,7.80952,7.656489e+03,6593.75279,3.704882e+03,5.740482e+03,0.00000,0.00000,0.10484,0.00000,0.00000,0.46342,0.00000,0.00000,0.01048,0.0,0.0,0.30821,0.00000,0.00000,0.26422,12.46735,12.46735,5.23755,1.12693,1.12693,0.46967,0.00000,0.00000,0.12790,1.49576,1.49576,0.59546,1.00000,1.00000,0.79047,0.00000,0.00000,0.02307,0.0,0.0,0.04823,1.00000,1.00000,0.59125,0.00000,0.00000,0.13000,0.00000,0.00000,0.40677,1.0,1.0,0.61840,1.0,1.0,0.40470,9.98463,9.98463,8.83154,0.00000,0.00000,5.51019
4,2020,00004B7C4023,TIN,474454561.0,RINDFLEISCH FAMILY PRACTICE PLLC,9335446202,ID-26820,"Molina Healthcare, Inc.",5.00000,57.99166,61.51546,5.388991e+04,49180.45071,9.815472e+04,7.204374e+04,1.00000,1.00000,1.41137,3.85948,3.85948,7.58740,1.00000,1.00000,0.23034,13.0,0.0,6.15417,2.66142,2.

In [7]:
conn = cb_utils.get_engine(source='msh_analytics')
table_name = f'{table_prefix}_data_tables_{year}_{current_folder}'
schema = 'raw'
print(f'select * from {schema}.{table_name}')
# pro_df.to_sql(pro_table_name, conn, schema=pro_schema, index=False, method='multi', chunksize=1000) # , if_exists='replace'

select * from raw.milliman_mani_data_tables_2020_20240812


In [9]:
mani_data_tables_2020_df.to_sql(table_name, conn, schema=schema, index=False, method='multi', chunksize=1000) # , if_exists='replace'

334212

In [10]:
file_name = 'Medicare Advantage Network Intelligence (MANI) CY2021 Data Tables - v2024.0 - Nationwide.csv'
year = 2021
mani_data_tables_2021_df = pd.read_csv(f'{data_dir}/{current_folder}/{file_name}',  encoding='latin1', low_memory=False)
# display(mani_data_tables_2020_df.head())
mani_data_tables_2021_df = cb_utils.df_format_columns(mani_data_tables_2021_df)
mani_data_tables_2021_df.head()

,ï»¿year,aco_builder_id,tin_ccn_type,tin_ccn_id,provider_name,pac_id,st_msa,carrier,py,risk_score_mmwtd,risk_score_mmwtd_region,ma_benchmark_rate,ma_bid_amount,paid_atffsreimb,paid_atffsreimb_region,ip_admits,ip_admits_imputed,ip_admits_region,ip_days,ip_days_imputed,ip_days_region,snf_admits,snf_admits_imputed,snf_admits_region,snf_days,snf_days_imputed,snf_days_region,ed,ed_imputed,ed_region,phy_office,phy_office_imputed,phy_office_region,rad_adv_imag_op,rad_adv_imag_op_imputed,rad_adv_imag_op_region,rad_adv_imag_nonhosp,rad_adv_imag_nonhosp_imputed,rad_adv_imag_nonhosp_region,rad_gen_op,rad_gen_op_imputed,rad_gen_op_region,rad_gen_nonhosp,rad_gen_nonhosp_imputed,rad_gen_nonhosp_region,rad_therap_op,rad_therap_op_imputed,rad_therap_op_region,rad_therap_nonhosp,rad_therap_nonhosp_imputed,rad_therap_nonhosp_region,prev_physicals,prev_physicals_imputed,prev_physicals_region,surgery_asc,surgery_asc_imputed,surgery_asc_region,surgery_op,surgery_op_imputed,surgery_op_region,part_b_rx_nonhosp,part_b_rx_nonhosp_imputed,part_b_rx_nonhosp_region,part_b_rx_op,part_b_rx_op_imputed,part_b_rx_op_region,lab_nonhosp,lab_nonhosp_imputed,lab_nonhosp_region,lab_op,lab_op_imputed,lab_op_region
0,2021,00004B7C4023,TIN,474454561.0,RINDFLEISCH FAMILY PRACTICE PLLC,9335446202,ID-26820,"Blue Cross of Idaho Health Services, Inc.",57.25000,668.87944,661.49104,6.544029e+05,574624.71150,6.725263e+05,6.818959e+05,17.84242,17.84242,11.64326,90.06221,35.06221,69.55973,0.0,0.0,1.88079,0.0,0.0,43.92307,90.00000,0.00000,31.76057,458.59408,5.59408,431.79256,47.0,0.0,43.25978,9.97145,9.97145,8.01355,72.00000,0.00000,65.61929,45.00000,0.00000,56.11757,0.0,0.0,4.74798,0.0,0.0,1.40504,20.30391,6.30391,30.12698,12.0,0.0,4.65597,13.32008,13.32008,31.10315,60.0,1.0,54.90814,2.0000,2.0000,69.91020,406.0,0.0,632.24219,87.00000,0.00000,461.80923
1,2021,00004B7C4023,TIN,474454561.0,RINDFLEISCH FAMILY PRACTICE PLLC,9335446202,ID-26820,CVS Health Corporation,2.00000,21.77081,22.40499,2.226602e+04,18505.30110,2.417759e+04,2.271020e+04,0.00000,0.00000,0.40084,0.00000,0.00000,1.39238,0.0,0.0,0.13186,0.0,0.0,3.42773,0.00000,0.00000,1.81421,3.24653,3.24653,10.13086,0.0,0.0,1.53442,0.00000,0.00000,0.27420,0.00000,0.00000,1.49231,0.00000,0.00000,0.59592,0.0,0.0,0.00000,0.0,0.0,0.00000,0.00000,0.00000,0.86499,0.0,0.0,0.20563,0.00000,0.00000,1.62964,0.0,0.0,1.13379,0.0000,0.0000,1.47675,135.0,0.0,26.33203,0.00000,0.00000,14.41173
2,2021,00004B7C4023,TIN,474454561.0,RINDFLEISCH FAMILY PRACTICE PLLC,9335446202,ID-26820,FFS,134.33333,1615.36737,1571.66835,1.588136e+06,0.00000,1.326359e+06,1.559343e+06,12.00000,0.00000,0.00000,126.00000,0.00000,0.00000,0.0,0.0,0.00000,40.0,0.0,0.00000,48.00000,0.00000,0.00000,874.00000,0.00000,0.00000,104.0,0.0,0.00000,0.00000,0.00000,0.00000,77.00000,0.00000,0.00000,44.00000,0.00000,0.00000,11.0,0.0,0.00000,0.0,0.0,0.00000,45.00000,0.00000,0.00000,0.0,0.0,0.00000,27.00000,0.00000,0.00000,96.0,0.0,0.00000,18.0000,0.0000,0.00000,3083.0,0.0,0.00000,277.00000,0.00000,0.00000
3,2021,00004B7C4023,TIN,474454561.0,RINDFLEISCH FAMILY PRACTICE PLLC,9335446202,ID-26820,Humana Inc.,4.08333,64.41775,62.56212,6.242565e+04,56951.49747,6.682203e+04,5.042460e+04,1.00000,1.00000,0.94727,3.63629,3.63629,5.31696,0.0,0.0,0.20159,0.0,0.0,5.00647,1.13549,1.13549,1.96416,54.00000,0.00000,31.62614,1.0,1.0,3.25092,3.12112,3.12112,0.95516,8.01766,8.01766,4.32982,1.94954,1.94954,3.95523,0.0,0.0,0.39076,0.0,0.0,0.24575,2.77182,2.77182,2.13201,0.0,0.0,0.35959,4.70198,4.70198,2.11033,1.0,1.0,3.91682,41.0000,1.0000,4.17636,115.0,0.0,40.23120,34.00000,0.00000,44.50382
4,2021,00004B7C4023,TIN,474454561.0,RINDFLEISCH FAMILY PRACTICE PLLC,9335446202,ID-26820,ID Other Carriers,2.00000,6.49522,7.23591,6.367185e+03,5406.79273,9.681091e+03,7.715731e+03,0.00000,0.00000,0.08188,0.00000,0.00000,0.38103,0.0,0.0,0.00529,0.0,0.0,0.10307,0.00000,0.00000,0.20421,9.22153,9.22153,7.53461,0.0,0.0,0.50977,0.00000,0.00000,0.15916,0.00000,0.00000,0.84320,1.00000,1.00000,1.27258,0.0,0.0,0.07919,0.0,0.0,0.

In [11]:
conn = cb_utils.get_engine(source='msh_analytics')
table_name = f'{table_prefix}_data_tables_{year}_{current_folder}'
schema = 'raw'
print(f'select * from {schema}.{table_name}')
# pro_df.to_sql(pro_table_name, conn, schema=pro_schema, index=False, method='multi', chunksize=1000) # , if_exists='replace'

select * from raw.milliman_mani_data_tables_2021_20240812


In [12]:
mani_data_tables_2021_df.to_sql(table_name, conn, schema=schema, index=False, method='multi', chunksize=1000) # , if_exists='replace'

354510

### Scorecard

In [13]:
file_name = 'Medicare Advantage Network Intelligence (MANI) CY2020 Scorecard - v2024.0 - Nationwide.xlsb'
year = 2020
score_card_2020_df = pd.read_excel(f'{data_dir}/{current_folder}/{file_name}')
score_card_2020_df = cb_utils.df_format_columns(score_card_2020_df)
print(f'{score_card_2020_df.shape[0]} rows, {score_card_2020_df.shape[1]} columns')
score_card_2020_df.head()

NameError: name 'retro_file_name' is not defined

In [ ]:
conn = cb_utils.get_engine(source='msh_analytics')
table_name = f'{table_prefix}_scorecard_{year}_{current_folder}'
schema = 'raw'
print(f'select * from {schema}.{table_name}')

In [ ]:
score_card_2020_df.to_sql(table_name, conn, schema=schema, index=False, method='multi', chunksize=1000) # , if_exists='replace'

In [ ]:
file_name = 'Medicare Advantage Network Intelligence (MANI) CY2021 Scorecard - v2024.0 - Nationwide.xlsb'
year = 2021
score_card_2021_df = pd.read_excel(f'{data_dir}/{current_folder}/{file_name}')
score_card_2021_df = cb_utils.df_format_columns(score_card_2021_df)
print(f'{score_card_2021_df.shape[0]} rows, {score_card_2021_df.shape[1]} columns')
score_card_2021_df.head()

In [ ]:
conn = cb_utils.get_engine(source='msh_analytics')
table_name = f'{table_prefix}_scorecard_{year}_{current_folder}'
schema = 'raw'
print(f'select * from {schema}.{table_name}')

In [ ]:
score_card_2020_df.to_sql(table_name, conn, schema=schema, index=False, method='multi', chunksize=1000) # , if_exists='replace'